# Cài đặt mã nguồn GAT




In [2]:
import torch
import os
import numpy as np

"""
Mở file entity2id ra và kết quả trả ra là một dictionary chứa key là entity, value là entity_id
""" 
def read_entity_from_id(filename='./data/WN18RR/entity2id.txt'):
    entity2id = {}
    with open(filename, 'r') as f:
        for line in f:
            if len(line.strip().split()) > 1:
                entity, entity_id = line.strip().split()[0].strip(), line.strip().split()[1].strip()
                entity2id[entity] = int(entity_id)
    return entity2id


"""
Tương tự như trên trả ra dictionary key, value
"""
def read_relation_from_id(filename='./data/WN18RR/relation2id.txt'):
    relation2id = {}
    with open(filename, 'r') as f:
        for line in f:
            if len(line.strip().split()) > 1:
                relation, relation_id = line.strip().split(
                )[0].strip(), line.strip().split()[1].strip()
                relation2id[relation] = int(relation_id)
    return relation2id

ModuleNotFoundError: No module named 'torch'

In [4]:
"""
Đọc file entity và relation, sau đó đọc từng dòng và trả ra mảng numpy kiểu float 32 các entity hoặc relation nhúng
"""
def init_embeddings(entity_file, relation_file):
    entity_emb, relation_emb = [], []

    with open(entity_file) as f:
        for line in f:
            entity_emb.append([float(val) for val in line.strip().split()])

    with open(relation_file) as f:
        # Đọc từng dòng
        for line in f:
            relation_emb.append([float(val) for val in line.strip().split()])

    return np.array(entity_emb, dtype=np.float32), np.array(relation_emb, dtype=np.float32)
"""
Kết quả trả ra là :
np.array(entity_emb, dtype=np.float32) : 
Là một mảng numpy kiểu float32 , giá trị là kết quả nhúng của thực thể được dòng từng dòng từ file

np.array(relation_emb, dtype=np.float32)
"""

'\nKết quả trả ra là :\nnp.array(entity_emb, dtype=np.float32) : \nLà một mảng numpy kiểu float32 , giá trị là kết quả nhúng của thực thể được dòng từng dòng từ file\n\nnp.array(relation_emb, dtype=np.float32)\n'

In [5]:
"""
Đọc từng dòng bộ ba :
<entity1> <realtion1> <entity1>
<entity2> <realtion2> <entity2>
...
"""
def parse_line(line):
    line = line.strip().split()
    e1, relation, e2 = line[0].strip(), line[1].strip(), line[2].strip()
    return e1, relation, e2

In [1]:
"""
rows, cols, data : là bộ ba gồm entity_head_id, entity_tail_id, relation
[rows, cols] = data
Vi dụ : head1_id relation1_id tail1_id
rows = [head1_id,...]
cols = [relation1_id,...]
data = [tail1_id,...]
"""
def load_data(filename, entity2id, relation2id, is_unweigted=False, directed=True):
    # Mở ra và đọc đến cuối
    with open(filename) as f:
        lines = f.readlines()

    # this is list for relation triples
    triples_data = []

    # for sparse tensor, rows list contains corresponding row of sparse tensor, cols list contains corresponding
    # columnn of sparse tensor, data contains the type of relation
    # Adjacecny matrix of entities is undirected, as the source and tail entities should know, the relation
    # type they are connected with
    rows, cols, data = [], [], []
    unique_entities = set()
    for line in lines:
        # Tách ra thành mối quan hệ
        e1, relation, e2 = parse_line(line)
        unique_entities.add(e1)
        unique_entities.add(e2)
        
        # Nhúng kết quả số hóa
        triples_data.append(
            (entity2id[e1], relation2id[relation], entity2id[e2]))
        if not directed:
                # Connecting source and tail entity
            rows.append(entity2id[e1])
            cols.append(entity2id[e2])
            if is_unweigted:
                data.append(1)
            else:
                data.append(relation2id[relation])

        # Connecting tail and source entity
        rows.append(entity2id[e2])
        cols.append(entity2id[e1])
        if is_unweigted:
            data.append(1)
        else:
            data.append(relation2id[relation])

    print("number of unique_entities ->", len(unique_entities))
    return triples_data, (rows, cols, data), list(unique_entities)

"""
triples_data : là bộ ba số hóa của head entity relation và tail entity
unique_entities : là tập hợp set entity
"""

'\ntriples_data : là bộ ba số hóa của head entity relation và tail entity\nunique_entities : là tập hợp set entity\n'

In [2]:
def build_data(path='./data/WN18RR/', is_unweigted=False, directed=True):
    entity2id = read_entity_from_id(path + 'entity2id.txt')
    relation2id = read_relation_from_id(path + 'relation2id.txt')

    # Adjacency matrix only required for training phase
    # Currenlty creating as unweighted, undirected
    train_triples, train_adjacency_mat, unique_entities_train = load_data(os.path.join(
        path, 'train.txt'), entity2id, relation2id, is_unweigted, directed)
    
    validation_triples, valid_adjacency_mat, unique_entities_validation = load_data(
        os.path.join(path, 'valid.txt'), entity2id, relation2id, is_unweigted, directed)
    
    test_triples, test_adjacency_mat, unique_entities_test = load_data(os.path.join(
        path, 'test.txt'), entity2id, relation2id, is_unweigted, directed)
    
    """
    Đên đây đã load ra được bộ ba gồm có train_triples, train_adjacency_mat, và unique_entities_train của cả 3
    """

    id2entity = {v: k for k, v in entity2id.items()}
    id2relation = {v: k for k, v in relation2id.items()}
    left_entity, right_entity = {}, {}
    # Trả ra diction với mỗi cái là id to entity hoặc relation

    with open(os.path.join(path, 'train.txt')) as f:
        lines = f.readlines()

    # Đọc với mỗi dòng
    for line in lines:
        e1, relation, e2 = parse_line(line)
        
        # Đếm số lượng quan hệ giữa (e1, relation) khi duyệt qua từng bộ ba
        # Count number of occurences for each (e1, relation)
        if relation2id[relation] not in left_entity:
            left_entity[relation2id[relation]] = {}
        if entity2id[e1] not in left_entity[relation2id[relation]]:
            left_entity[relation2id[relation]][entity2id[e1]] = 0
        left_entity[relation2id[relation]][entity2id[e1]] += 1

        # Count number of occurences for each (relation, e2)
        if relation2id[relation] not in right_entity:
            right_entity[relation2id[relation]] = {}
        if entity2id[e2] not in right_entity[relation2id[relation]]:
            right_entity[relation2id[relation]][entity2id[e2]] = 0
        right_entity[relation2id[relation]][entity2id[e2]] += 1
        
    """
    Đến đây ta đã tính đươc tổng số hàng xóm liên kết dến head hoặc tail trong một node của head entity và tail entity
    num_neighbour(head) = left_entity
    num_neighbour(tail) = right_entity
    
    Trong đó, kết quả dữ liệu sẽ có dạng
    left_entity[relation_id][head_id] = tổng số lượng hàng xóm của head
    right_entity[relation_id][tail_id] = tổng số lượng hàng xóm của tail
    """
    
    
    left_entity_avg = {}
    for i in range(len(relation2id)):
        left_entity_avg[i] = sum(
            left_entity[i].values()) * 1.0 / len(left_entity[i])
    """Với mỗi quan hệ realtion_i (trong tất cả quan hệ có relation_i) : tính trung bình số hàng xóm của phần head"""
        

    right_entity_avg = {}
    for i in range(len(relation2id)):
        right_entity_avg[i] = sum(
            right_entity[i].values()) * 1.0 / len(right_entity[i])
    """Với mỗi quan hệ realtion : tính trung bình số hàng xóm của tail"""

    """
    Duyệt tất cả trong tập realtions
    headTailSelector = relation_i_th = 1000 * trung_bình_số_neibour_tail_i / (số node head + tail)
    
    => Đây là một hàm tính điểm số
    """
    headTailSelector = {}
    for i in range(len(relation2id)):
        headTailSelector[i] = 1000 * right_entity_avg[i] / \
            (right_entity_avg[i] + left_entity_avg[i])

    return (train_triples, train_adjacency_mat), (validation_triples, valid_adjacency_mat), (test_triples, test_adjacency_mat), \
        entity2id, relation2id, headTailSelector, unique_entities_train

"""
Kết quả trả ra là :
(train_triples, train_adjacency_mat) : tập train
(validation_triples, valid_adjacency_mat) : tập valid
(test_triples, test_adjacency_mat) : tập test
entity2id : entity_id
relation2id : relation_id
headTailSelector : độ đo số node của tail trên tổng số hàng xóm của 1 relation
unique_entities_train : unique
"""

''

# Hàm thực thi quá trình trên
train_data, validation_data, test_data, entity2id, relation2id, headTailSelector, unique_entities_train = build_data(
        args.data, is_unweigted=False, directed=True)

In [ ]:
def load_data(args):
    """
    Tương tự quá trình trên"""
    train_data, validation_data, test_data, entity2id, relation2id, headTailSelector, unique_entities_train = build_data(
        args.data, is_unweigted=False, directed=True)

    if args.pretrained_emb:
        entity_embeddings, relation_embeddings = init_embeddings(os.path.join(args.data, 'entity2vec.txt'),
                                                                 os.path.join(args.data, 'relation2vec.txt'))
        print("Initialised relations and entities from TransE")

    else:
        """
        Khởi tạo entity_embeddings với kích thước len(entity2id) và kích thước nhúng
        Tức số vector biểu diễn 1 embed trong không gian
        """
        entity_embeddings = np.random.randn(
            len(entity2id), args.embedding_size)
        relation_embeddings = np.random.randn(
            len(relation2id), args.embedding_size)
        print("Initialised relations and entities randomly")

    corpus = Corpus(args, train_data, validation_data, test_data, entity2id, relation2id, headTailSelector,
                    args.batch_size_gat, args.valid_invalid_ratio_gat, unique_entities_train, args.get_2hop)

    return corpus, torch.FloatTensor(entity_embeddings), torch.FloatTensor(relation_embeddings)

In [4]:
def get_graph(self):
        graph = {}
        all_tiples = torch.cat([self.train_adj_matrix[0].transpose(0, 1), self.train_adj_matrix[1].unsqueeze(1)], dim=1)
        """
        train_adj_matrix = [[tail_id, head_id], [relation]]
        Chuyển từ dạng [[tail_id, head_id], [relation]] sang dạng (n dòng, 1 cột) [[tail_id], [head_id], [relation_id]]
        """

        for data in all_tiples:
            source = data[1].data.item() # head
            target = data[0].data.item() # tail
            value = data[2].data.item() # relation

            if(source not in graph.keys()):
                graph[source] = {}
                graph[source][target] = value
            else:
                graph[source][target] = value
        print("Graph created")
        return graph
    
"""
Tạo ra kết quả bao gồm một dictionary gồm
graph[head_id][tail_id] = relation_id

=> Tức là lấy ra tất cả các cạnh trong đồ thị
"""

In [ ]:
"""
Mục tiêu : Dùng thuật toán BFS để tìm những node xung quanh với 1 node source với độ sâu là 2
Input : 
graph : chứa tất cả các cạnh có hướng trong đồ thị với cấu trúc : [[head_id], [tail_id], [relation_id]]
source : node nguồn
nbd_size : độ sâu của node
"""
def bfs(self, graph, source, nbd_size=2):
        visit = {} # Tập hợp đã ghé thăm
        distance = {} # Khoảng cách
        parent = {} # Tập hợp node cha
        distance_lengths = {} # Chiều dài khoảng cách

        visit[source] = 1 # Ghé thăm node gốc
        distance[source] = 0 # Khoảng cách đến node đang xét tới tất cả các node
        parent[source] = (-1, -1) # Cha của node gốc là không có ai cả

        q = queue.Queue() # Queue
        q.put((source, -1)) # Đưa node gốc vào

        while(not q.empty()): # Khi Queue vẫn còn rỗng
            top = q.get() # Lấy phần tử đầu tiên ra
            if top[0] in graph.keys(): # nếu node head đang duyệt nằm trong đồ thị
                for target in graph[top[0]].keys(): # node tail nằm trong đồ thị
                    if(target in visit.keys()): # nếu tail đã nằm trong danh sách duyệt rồi thì đi tiếp
                        continue
                    else:
                        q.put((target, graph[top[0]][target]))

                        distance[target] = distance[top[0]] + 1

                        visit[target] = 1
                        if distance[target] > 2:
                            continue
                        parent[target] = (top[0], graph[top[0]][target])

                        if distance[target] not in distance_lengths.keys():
                            distance_lengths[distance[target]] = 1

        neighbors = {}
        for target in visit.keys():
            if(distance[target] != nbd_size):
                continue
            edges = [-1, parent[target][1]]
            relations = []
            entities = [target]
            temp = target
            while(parent[temp] != (-1, -1)):
                relations.append(parent[temp][1])
                entities.append(parent[temp][0])
                temp = parent[temp][0]

            if(distance[target] in neighbors.keys()):
                neighbors[distance[target]].append(
                    (tuple(relations), tuple(entities[:-1])))
            else:
                neighbors[distance[target]] = [
                    (tuple(relations), tuple(entities[:-1]))]

        return neighbors
    
"""
Kết quả trả ra là một dictionary chứa
neighbors[distance] = (relations_id, entities_id)
"""

In [6]:
"""
nbd_size : kích thước độ sâu khi duyệt hàng xóm kế cận
"""
def get_further_neighbors(self, nbd_size=2):
        neighbors = {}
        start_time = time.time()
        print("length of graph keys is ", len(self.graph.keys()))
        
        # Với mỗi head trong từng cặp head, relation, tail trong tập train
        for source in self.graph.keys():
            # st_time = time.time()
            
            """
            Trả ra những node hàng xóm với độ sâu là 2
            temp_neighbors có định dạng :
            temp_neighbors[distance] = (relations_id, entities_id)
            
            neighbors[source_id][distance] = (relations_id, entity_id)
            """ 
            temp_neighbors = self.bfs(self.graph, source, nbd_size)
            for distance in temp_neighbors.keys():
                if(source in neighbors.keys()):
                    if(distance in neighbors[source].keys()):
                        neighbors[source][distance].append(
                            temp_neighbors[distance])
                    else:
                        neighbors[source][distance] = temp_neighbors[distance]
                else:
                    neighbors[source] = {}
                    neighbors[source][distance] = temp_neighbors[distance]

        print("time taken ", time.time() - start_time)

        print("length of neighbors dict is ", len(neighbors))
        return neighbors

SyntaxError: invalid syntax (<ipython-input-6-06ac2e8a9b1b>, line 13)

In [7]:
class Corpus:
    def __init__(self, args, train_data, validation_data, test_data, entity2id,
                 relation2id, headTailSelector, batch_size, valid_to_invalid_samples_ratio, unique_entities_train, get_2hop=False):
        self.train_triples = train_data[0]
        

        # Converting to sparse tensor
        """Lấy từ train_adjacency_mat
        adj_indices sẽ có dạng là tuple (rows, cols, data)
        row = [head1,..]
        cols = [tail1,..]
        data = [relation1,..]
        """
        adj_indices = torch.LongTensor(
            [train_data[1][0], train_data[1][1]])  # rows and columns
        # data = train_data[1][2] với train_data[1] = (rows, cols, data)
        adj_values = torch.LongTensor(train_data[1][2])
        self.train_adj_matrix = (adj_indices, adj_values)
        # train_adj_matrix : ([head_id, tail_id], [relation]

        # adjacency matrix is needed for train_data only, as GAT is trained for
        # training data
        self.validation_triples = validation_data[0]
        self.test_triples = test_data[0]

        self.headTailSelector = headTailSelector  # for selecting random entities
        # Với mỗi quan hệ, tính trung bình tổng số hàng xóm tail trên tổng số hàng xóm, có quan hệ
        self.entity2id = entity2id
        self.id2entity = {v: k for k, v in self.entity2id.items()}
        self.relation2id = relation2id
        self.id2relation = {v: k for k, v in self.relation2id.items()}
        self.batch_size = batch_size
        # ratio of valid to invalid samples per batch for training ConvKB Model
        self.invalid_valid_ratio = int(valid_to_invalid_samples_ratio)

        if(get_2hop):
            self.graph = self.get_graph()
            """
            Tạo ra kết quả bao gồm một dictionary gồm
            self.graph[head_id][tail_id] = relation_id
            """
            self.node_neighbors_2hop = self.get_further_neighbors()

        self.unique_entities_train = [self.entity2id[i]
                                      for i in unique_entities_train]
        # unique_entities_train : danh sách entity_id trong tập train

        # train_indices có shape là [[head_id, tail_id, relation_id]]
        self.train_indices = np.array(
            list(self.train_triples)).astype(np.int32)
        
        # These are valid triples, hence all have value 1
        # Kích thước là : 1 x số cạnh trong đồ thị, với giá trị với cái là 1
        self.train_values = np.array(
            [[1]] * len(self.train_triples)).astype(np.float32)

        self.validation_indices = np.array(
            list(self.validation_triples)).astype(np.int32)
        self.validation_values = np.array(
            [[1]] * len(self.validation_triples)).astype(np.float32)

        self.test_indices = np.array(list(self.test_triples)).astype(np.int32)
        self.test_values = np.array(
            [[1]] * len(self.test_triples)).astype(np.float32)
        
        """
        Tương tự với tập valid và tập test
        """

        self.valid_triples_dict = {j: i for i, j in enumerate(
            self.train_triples + self.validation_triples + self.test_triples)}
        print("Total triples count {}, training triples {}, validation_triples {}, test_triples {}".format(len(self.valid_triples_dict), len(self.train_indices),
                                                                                                           len(self.validation_indices), len(self.test_indices)))

        # For training purpose
        """
        batch_indices : có kích thước là (batch_invalid_size, 3) : kiểu int32
        Ý nghĩa : là tập để chọn ra danh sách những tập valid, phân biệt với tập invalid trong đồ thị
        
        batch_values có kích thước chỉ là : (batch_invalid_size, 3)
        Dùng để biểu diễn giá trị
        """
        self.batch_indices = np.empty(
            (self.batch_size * (self.invalid_valid_ratio + 1), 3)).astype(np.int32)
        self.batch_values = np.empty(
            (self.batch_size * (self.invalid_valid_ratio + 1), 1)).astype(np.float32)

In [8]:
def get_iteration_batch(self, iter_num):
        if (iter_num + 1) * self.batch_size <= len(self.train_indices):
            self.batch_indices = np.empty(
                (self.batch_size * (self.invalid_valid_ratio + 1), 3)).astype(np.int32)
            self.batch_values = np.empty(
                (self.batch_size * (self.invalid_valid_ratio + 1), 1)).astype(np.float32)

            indices = range(self.batch_size * iter_num,
                            self.batch_size * (iter_num + 1))

            self.batch_indices[:self.batch_size,
                               :] = self.train_indices[indices, :]
            self.batch_values[:self.batch_size,
                              :] = self.train_values[indices, :]

            last_index = self.batch_size

            if self.invalid_valid_ratio > 0:
                random_entities = np.random.randint(
                    0, len(self.entity2id), last_index * self.invalid_valid_ratio)

                # Precopying the same valid indices from 0 to batch_size to rest
                # of the indices
                self.batch_indices[last_index:(last_index * (self.invalid_valid_ratio + 1)), :] = np.tile(
                    self.batch_indices[:last_index, :], (self.invalid_valid_ratio, 1))
                self.batch_values[last_index:(last_index * (self.invalid_valid_ratio + 1)), :] = np.tile(
                    self.batch_values[:last_index, :], (self.invalid_valid_ratio, 1))

                for i in range(last_index):
                    for j in range(self.invalid_valid_ratio // 2):
                        current_index = i * (self.invalid_valid_ratio // 2) + j

                        while (random_entities[current_index], self.batch_indices[last_index + current_index, 1],
                               self.batch_indices[last_index + current_index, 2]) in self.valid_triples_dict.keys():
                            random_entities[current_index] = np.random.randint(
                                0, len(self.entity2id))
                        self.batch_indices[last_index + current_index,
                                           0] = random_entities[current_index]
                        self.batch_values[last_index + current_index, :] = [-1]

                    for j in range(self.invalid_valid_ratio // 2):
                        current_index = last_index * \
                            (self.invalid_valid_ratio // 2) + \
                            (i * (self.invalid_valid_ratio // 2) + j)

                        while (self.batch_indices[last_index + current_index, 0], self.batch_indices[last_index + current_index, 1],
                               random_entities[current_index]) in self.valid_triples_dict.keys():
                            random_entities[current_index] = np.random.randint(
                                0, len(self.entity2id))
                        self.batch_indices[last_index + current_index,
                                           2] = random_entities[current_index]
                        self.batch_values[last_index + current_index, :] = [-1]

                return self.batch_indices, self.batch_values

            return self.batch_indices, self.batch_values

        else:
            last_iter_size = len(self.train_indices) - \
                self.batch_size * iter_num
            self.batch_indices = np.empty(
                (last_iter_size * (self.invalid_valid_ratio + 1), 3)).astype(np.int32)
            self.batch_values = np.empty(
                (last_iter_size * (self.invalid_valid_ratio + 1), 1)).astype(np.float32)

            indices = range(self.batch_size * iter_num,
                            len(self.train_indices))
            self.batch_indices[:last_iter_size,
                               :] = self.train_indices[indices, :]
            self.batch_values[:last_iter_size,
                              :] = self.train_values[indices, :]

            last_index = last_iter_size

            if self.invalid_valid_ratio > 0:
                random_entities = np.random.randint(
                    0, len(self.entity2id), last_index * self.invalid_valid_ratio)

                # Precopying the same valid indices from 0 to batch_size to rest
                # of the indices
                self.batch_indices[last_index:(last_index * (self.invalid_valid_ratio + 1)), :] = np.tile(
                    self.batch_indices[:last_index, :], (self.invalid_valid_ratio, 1))
                self.batch_values[last_index:(last_index * (self.invalid_valid_ratio + 1)), :] = np.tile(
                    self.batch_values[:last_index, :], (self.invalid_valid_ratio, 1))

                for i in range(last_index):
                    for j in range(self.invalid_valid_ratio // 2):
                        current_index = i * (self.invalid_valid_ratio // 2) + j

                        while (random_entities[current_index], self.batch_indices[last_index + current_index, 1],
                               self.batch_indices[last_index + current_index, 2]) in self.valid_triples_dict.keys():
                            random_entities[current_index] = np.random.randint(
                                0, len(self.entity2id))
                        self.batch_indices[last_index + current_index,
                                           0] = random_entities[current_index]
                        self.batch_values[last_index + current_index, :] = [-1]

                    for j in range(self.invalid_valid_ratio // 2):
                        current_index = last_index * \
                            (self.invalid_valid_ratio // 2) + \
                            (i * (self.invalid_valid_ratio // 2) + j)

                        while (self.batch_indices[last_index + current_index, 0], self.batch_indices[last_index + current_index, 1],
                               random_entities[current_index]) in self.valid_triples_dict.keys():
                            random_entities[current_index] = np.random.randint(
                                0, len(self.entity2id))
                        self.batch_indices[last_index + current_index,
                                           2] = random_entities[current_index]
                        self.batch_values[last_index + current_index, :] = [-1]

                return self.batch_indices, self.batch_values

            return self.batch_indices, self.batch_values